In [8]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt



In [9]:
def get_concacted_df_from_folder(folder_path):
    csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

    # Read and concatenate all CSV files into one DataFrame
    df_list = [pd.read_csv(os.path.join(folder_path, file)) for file in csv_files]
    concatenated_df = pd.concat(df_list, ignore_index=True)
    return concatenated_df

def get_i_df_from_folder(folder_path, i):
    csv_file = [f for f in os.listdir(folder_path) if f.endswith('.csv')][i]

    # Read and concatenate all CSV files into one DataFrame
    df = pd.read_csv(os.path.join(folder_path, csv_file))
    return df


In [10]:
GPT_ANSWER_FOLDER = 'responses/p2'
ORIGINAL_FILES_FOLDER = 'original_people'

#df = get_concacted_df_from_folder(GPT_ANSWER_FOLDER)
df = get_i_df_from_folder(ORIGINAL_FILES_FOLDER, 0)

exclude_columns = ['Name', 'Score']  

for column in df.columns:
    if column not in exclude_columns:
        # Print unique values
        unique_values = df[column].unique()
        print(f"Unique values in '{column}': {sorted(unique_values, reverse=True)}")  # Sorted in descending order
        
        # Print counts for each value, sorted from most to less
        value_counts = df[column].value_counts().sort_values(ascending=False)
        print(f"Counts per {column}:\n{value_counts}\n")
        
        # Calculate and print average score per category within the column, sorted from highest to lowest
        average_score_per_category = df.groupby(column)['Score'].mean().sort_values(ascending=False)
        print(f"Average Score per {column}:\n{average_score_per_category}\n")
        
        # Plotting average score per category, sorted
        plt.figure(figsize=(10, 6))
        average_score_per_category.sort_values().plot(kind='barh')  # Horizontal bar plot for better readability
        plt.title(f'Average Score per {column}')
        plt.xlabel('Average Score')
        plt.ylabel(column)
        plt.tight_layout()
        plt.show()

Unique values in 'Gender': ['Other', 'Male', 'Female']
Counts per Gender:
Gender
Female    3371
Other     3335
Male      3294
Name: count, dtype: int64



KeyError: 'Column not found: Score'